In [11]:
import requests

In [13]:
total_edicoes = 2220
url_base = 'https://revistas.inpi.gov.br/txt/RM{}.zip'

# Loop de download das revistas antigas
for index in reversed(range(1, total_edicoes + 1)):
    # Substitui o índice na URL
    url_api = url_base.format(index)
    response = requests.get(url_api)
    
    # Verifica se o texto de erro está no conteúdo da resposta
    if "A URL requisitada" in response.text:
        print(f"Revista {index} não encontrada.")
    else:
        print(f"Revista {index} baixada com sucesso.")

Revista 2220 baixada com sucesso.
Revista 2219 baixada com sucesso.
Revista 2218 baixada com sucesso.
Revista 2217 baixada com sucesso.
Revista 2216 baixada com sucesso.
Revista 2215 baixada com sucesso.
Revista 2214 baixada com sucesso.
Revista 2213 baixada com sucesso.
Revista 2212 baixada com sucesso.
Revista 2211 baixada com sucesso.
Revista 2210 baixada com sucesso.
Revista 2209 baixada com sucesso.


KeyboardInterrupt: 

In [21]:
import requests
import zipfile
import os
import sqlite3
from io import BytesIO
from bs4 import BeautifulSoup
import pandas as pd

# Configuração do banco de dados
DATABASE = 'revistas-test.db'

def create_database():
    print('Iniciando a criação do banco.')
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()
    
    # Criação da tabela com todos os campos necessários
    cursor.execute("""CREATE TABLE revistas (
                        numero_revista TEXT,
                        data_revista TEXT,
                        numero_processo TEXT,
                        data_deposito TEXT,
                        data_concessao TEXT,
                        data_vigencia TEXT,
                        despachos TEXT,
                        titulares TEXT,
                        marca_nome TEXT,
                        marca_apresentacao TEXT,
                        marca_natureza TEXT,
                        classes_vienna TEXT,
                        lista_classe_nice TEXT,
                        procurador TEXT
                      )""")
    
    conn.commit()
    conn.close()
    print('Banco de dados criado!')

# Exemplo de uso
create_database()


Iniciando a criação do banco.
Banco de dados criado!


In [18]:
def is_new_revista(numero):
    print('Checando se é uma nova revista.')
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()
    cursor.execute("SELECT numero FROM revistas WHERE numero = ?", (numero))
    result = cursor.fetchone()
    conn.close()
    print('Revista checada!')
    return result is None


In [19]:
def fetch_and_download_zip(url_zip, numero_revista):
    print('Iniciando a download do arquivo.')
    response = requests.get(url_zip)
    if response.status_code == 200:
        print('Baixando...')
        zip_file = zipfile.ZipFile(BytesIO(response.content))
        # Cria um diretório para armazenar os arquivos baixados
        os.makedirs(f'revistas/{numero_revista}', exist_ok=True)
        zip_file.extractall(f'revistas/{numero_revista}')
        print('Baixado e inserido em revistas!')
        return f'revistas/{numero_revista}'
    else:
        print("Erro ao baixar o arquivo ZIP.")
        return None


In [20]:
import os
import pandas as pd
import xml.etree.ElementTree as ET
import json

def parse_revista(xml_string):
    revista_data = {
        "numero": "",
        "data": "",
        "processos": []
    }

    root = ET.fromstring(xml_string)
    revista_data["numero"] = root.get("numero", "")
    revista_data["data"] = root.get("data", "")

    for processo in root.findall("processo"):
        processo_data = {
            "numero": processo.get("numero", ""),
            "data_deposito": processo.get("data-deposito", ""),
            "data_concessao": processo.get("data-concessao", ""),
            "data_vigencia": processo.get("data-vigencia", ""),
            "despachos": [],
            "titulares": [],
            "marca": {
                "nome": "",
                "apresentacao": "",
                "natureza": ""
            },
            "classes_vienna": [],
            "lista_classe_nice": [],
            "procurador": ""
        }

        # Mapeia os despachos
        for despacho in processo.findall("despachos/despacho"):
            despacho_data = {
                "codigo": despacho.get("codigo", ""),
                "nome": despacho.get("nome", "")
            }
            processo_data["despachos"].append(despacho_data)

        # Mapeia os titulares
        for titular in processo.findall("titulares/titular"):
            titular_data = {
                "nome_razao_social": titular.get("nome-razao-social", ""),
                "pais": titular.get("pais", ""),
                "uf": titular.get("uf", "")
            }
            processo_data["titulares"].append(titular_data)

        # Mapeia a marca, se existir
        marca = processo.find("marca")
        if marca is not None:
            processo_data["marca"]["nome"] = marca.findtext("nome", "").strip()
            processo_data["marca"]["apresentacao"] = marca.get("apresentacao", "")
            processo_data["marca"]["natureza"] = marca.get("natureza", "")

        # Mapeia as classes de Viena, se existirem
        for classe in processo.findall("classes-vienna/classe-vienna"):
            classe_data = {
                "codigo": classe.get("codigo", ""),
                "edicao": classe.get("edicao", "")
            }
            processo_data["classes_vienna"].append(classe_data)

        # Mapeia a lista de classe Nice
        for classe_nice in processo.findall("lista-classe-nice/classe-nice"):
            classe_nice_data = {
                "codigo": classe_nice.get("codigo", ""),
                "especificacao": classe_nice.findtext("especificacao", "").strip(),
                "status": classe_nice.findtext("status", "").strip()
            }
            processo_data["lista_classe_nice"].append(classe_nice_data)

        # Mapeia o procurador, se existir
        procurador = processo.find("procurador")
        processo_data["procurador"] = procurador.text.strip() if procurador is not None else ""

        # Adiciona o processo mapeado à lista de processos
        revista_data["processos"].append(processo_data)

    return revista_data

def process_xml_file(directory):
    all_data = []  # Lista para armazenar dados de todos os arquivos

    for file_name in os.listdir(directory):
        if file_name.endswith('.xml'):
            file_path = os.path.join(directory, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:
                xml_string = file.read()  # Ler o conteúdo do arquivo XML
                revista_data = parse_revista(xml_string)  # Parse do XML

                # Adicionar dados da revista ao resultado
                for processo in revista_data['processos']:
                    all_data.append({
                        'numero_revista': revista_data['numero'],
                        'data_revista': revista_data['data'],
                        'numero_processo': processo['numero'],
                        'data_deposito': processo['data_deposito'],
                        'data_concessao': processo['data_concessao'],
                        'data_vigencia': processo['data_vigencia'],
                        'despachos': json.dumps(processo['despachos']),
                        'titulares': json.dumps(processo['titulares']),
                        'marca_nome': processo['marca']['nome'],
                        'marca_apresentacao': processo['marca']['apresentacao'],
                        'marca_natureza': processo['marca']['natureza'],
                        'classes_vienna': json.dumps(processo['classes_vienna']),
                        'lista_classe_nice': json.dumps(processo['lista_classe_nice']),
                        'procurador': processo['procurador'],
                    })

    # Converte para DataFrame e retorna
    if all_data:  # Verifica se há dados coletados
        return pd.DataFrame(all_data)

    return None  # Retorna None se não houver arquivos XML ou dados


In [5]:
def insert_data_into_database(dataframe, numero_revista, data_revista, url_zip):
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()

    # Adicionar o número da revista à tabela de revistas
    cursor.execute("INSERT INTO revistas (numero, data, url_zip) VALUES (?, ?, ?)", (numero_revista, data_revista, url_zip))
    
    # Inserir os dados do XML processado
    dataframe.to_sql('dados_xml', conn, if_exists='append', index=False)

    conn.commit()
    conn.close()


In [13]:
def main():
    create_database()

    # URL da API (ou faça a requisição diretamente com web scraping)
    url_api = 'http://localhost:3000/api/scraping'
    response = requests.get(url_api)

    if response.status_code == 200:
        revistas = response.json()

        for revista in revistas:
            numero_revista = revista['numeroRevista']
            data_revista = revista['dataRevista']
            url_zip = revista['secaoMarcas'] # Ajuste para a seção correta que contém o link .zip

            if is_new_revista(numero_revista):
                print(f'Nova revista encontrada: {numero_revista}')

                # Baixar o arquivo ZIP
                download_directory = fetch_and_download_zip(url_zip, numero_revista)
                if download_directory:
                    # Processar o XML
                    df = process_xml_file(download_directory)
                    if df is not None:
                        # Inserir os dados no banco de dados
                        insert_data_into_database(df, numero_revista, data_revista, url_zip)
                        print(f'Revista {numero_revista} importada com sucesso.')
                    else:
                        print(f'Erro ao processar o XML para a revista {numero_revista}.')
            else:
                print(f'A revista {numero_revista} já foi processada.')
    else:
        print("Erro ao acessar a API.")


In [14]:
if __name__ == '__main__':
    main()


Iniciando a criação do banco.
Banco de dados criado!
Checando se é uma nova revista.


OperationalError: no such column: numero